<a href="https://colab.research.google.com/github/dayoungkimm/MS-AI-School/blob/main/PythonAzure01_StorageBlob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###스토리지 계정 생성 후 Azure Blob에 컨테이너 생성 및 이미지 업로드

In [2]:
!pip install azure-storage-blob azure-identity  #blob storage패키지 설치/권한 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.3 MB/s eta 0:00:00


In [3]:
import os, uuid 
from azure.identity import DefaultAzureCredential 
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

1. Blob Serivces에 접속

In [4]:
connect_str = 'DefaultEndpointsProtocol=https;AccountName=labuser6storage;AccountKey=LoCbaZ2IR0K3ExdWWckkdTVZrGk/8f2mGCk5sTtJCvf7WInW7TnIINyNufkTtW5K+tdQKToP0qDO+AStvxTNFw==;EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)    #연결 명령

2. container의 이름 짓기

In [5]:
container_name = 'test2container'

container_client = blob_service_client.create_container(container_name)

3. 코랩에 파일 생성하기

In [6]:
# Create a local directory to hold blob data
local_path = './data'   #현재 로컬 밑에 data라는 경로를 만들겠다.
os.mkdir(local_path)          #make 디렉토리 /디렉토리 생성(data 경로 생성)

# Create a file in the local data directory to upload and download
local_file_name = 'welcome' + '.txt'    #파일 이름
upload_file_path = os.path.join(local_path, local_file_name)  #파일 전체 경로

#Write text to the file
file = open(upload_file_path, mode='w')
file.write('Welcome to Python')
file.close()


In [7]:
blob_client = blob_service_client.get_blob_client(container = container_name,
                                                       blob = local_file_name)

with open(file=upload_file_path, mode='rb') as data:    #코랩 파일을 오픈한다./모드'rb'로 하면 binary단위>> 업로드할때 깨지지 않음!
  blob_client.upload_blob(data)

4. blob 파일의 리스트를 확인한다.

In [8]:
# blob 파일의 리스트를 확인한다.
print('\nListing blobs')

blob_list = container_client.list_blobs()
blob_list


Listing blobs


<iterator object azure.core.paging.ItemPaged at 0x7f471eee1040>

In [9]:
print('\nListing blobs')

blob_list = container_client.list_blobs()
for blob in blob_list:
  print('\t' + blob.name)


Listing blobs
	welcome.txt


5. 파일 다운받기


In [10]:
#Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extension so that you can cehck both files in the data directory

download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name,'.txt','_DOWNLOAD.txt'))   #파일명 교체
print(download_file_path)                           


./data/welcome_DOWNLOAD.txt


In [11]:
#파일 다운받기
download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name,'.txt','_DOWNLOAD.txt'))   #파일명 교체
print(download_file_path)                           
container_client = blob_service_client.get_container_client(container=container_name)

with open(file=download_file_path, mode='wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall())    #파일 다운받고/ readall 전체 읽어들이기기          


./data/welcome_DOWNLOAD.txt


6. 파일 삭제/ 리소스 정리

In [12]:
#파일 삭제/리소스 정리
print('\nPress the Enter Key to begin clean up')
input()            #if input() == 'yes': >> yes를 치면 삭제 / input은 엔터가 디폴트키!

print('Deleting blob container...')
container_client.delete_container()

print('Detleting the local source and downloaded files...')
os.remove(upload_file_path)
os.remove(download_file_path)
os.rmdir(local_path)        #remove directory

print('Done')


Press the Enter Key to begin clean up

Deleting blob container...
Detleting the local source and downloaded files...
Done
